In [175]:
from tinysmpc.tinysmpc.tinysmpc import VirtualMachine, PrivateScalar, SharedScalar
from tinysmpc.tinysmpc.secret_sharing import Share
from collections import defaultdict, deque

In [176]:
def secure_bfs_cycle_detection_forwarding(edge_list, destination_node):

    # Build a reversed adjacency list for the graph
    tmp_vm = VirtualMachine('tmp_vm')
    rev_adj_list = defaultdict(list)
    incoming_count = defaultdict(int)

    for u, v in edge_list:
        rev_adj_list[v].append(u)
        incoming_count[u] += 1

    queue = [destination_node]
    while queue:
        current_node = queue.pop(0)

        for neighbor in rev_adj_list[current_node]:
            incoming_count[neighbor] -= 1
            if incoming_count[neighbor] == 0:
                queue.append(neighbor)

    return PrivateScalar(int(any(incoming_count[node] > 0 for node in rev_adj_list)), tmp_vm)

In [177]:
edges = [
    (1,2),
    (2,3),
    (3,1),
    (3,4),
]
q = VirtualMachine("q")
n1 = VirtualMachine("n1")
n2 = VirtualMachine("n2")
n3 = VirtualMachine("n3")
n4 = VirtualMachine("n4")
v1 = PrivateScalar(1, n1)
v2 = PrivateScalar(2, n2)
v3 = PrivateScalar(3, n3)
v4 = PrivateScalar(1, n4)

In [178]:
s1 = v1.share([n1, n2, n3, n4])
s2 = v2.share([n1, n2, n3, n4])
s3 = v3.share([n1, n2, n3, n4])
s4 = v4.share([n1, n2, n3, n4])

In [179]:
s_edges = [
    (s1,s2),
    (s2,s3),
    (s3,s1),
    (s3,s4),
]

In [180]:
r_s = secure_bfs_cycle_detection_forwarding(s_edges, s4).share([n1, n2, n3, n4])
r_s.reconstruct(q)

PrivateScalar(1, 'q')